In [ ]:
%matplotlib notebook
from rfsoc_rfdc.rfsoc_overlay import RFSoCOverlay
from rfsoc_rfdc.overlay_task import OverlayTask
from rfsoc_rfdc.overlay_task import BlinkLedTask

from rfsoc_rfdc.transmitter.single_ch_tx_task import SingleChTxTask
from rfsoc_rfdc.receiver.single_ch_rx_task import SingleChRxTask

from rfsoc_rfdc.transmitter.multi_ch_tx_task import MultiChTxTask
from rfsoc_rfdc.receiver.multi_ch_rx_task import MultiChRxTask

from rfsoc_rfdc.rfdc_task import RfdcTask 
from rfsoc_rfdc.cmac_task import CmacTask
from rfsoc_rfdc.rfdc_config import ZCU216_CONFIG

import sys
import os
import time

In [ ]:
# RFSoC experiment configuration
# +----------------------------------------------------------------------------------------+
# | Feature: Single Channel 1.25 GHz BW Tx/Rx Experiment                                   |
# +-----------------+--------------------+----------------+-----------+-------+------------+
# | DAC Sample Rate | Interpolation Rate | IQ Sample Rate | Bandwidth | NCO   | Ref. Clock |
# | 2.5 GSPS        | 2x                 | 1.25 GSPS      | 1.25G     | 625M  | 500M (PLL) |
# +-----------------+--------------------+----------------+-----------+-------+------------+
# | ADC Sample Rate | Decimation Rate    | IQ Sample Rate | Bandwidth | NCO   | Ref. Clock |
# | 2.5 GSPS        | 2x                 | 1.25 GSPS      | 1.25G     | -625M | 500M (PLL) |
# +-----------------+--------------------+----------------+-----------+-------+------------+

In [ ]:
ol = RFSoCOverlay(path_to_bitstream="./rfsoc_rfdc/bitstream/rfsoc_rfdc_v39.bit")
NEW_CONFIG = {
    "RefClockForPLL": 500.0,
    "DACSampleRate": 2500.0,
    "DACInterpolationRate": 2,
    "DACNCO": 625.0,
    "ADCSampleRate": 2500.0,
    "ADCInterpolationRate": 2,
    "ADCNCO": -625.0,
}
ZCU216_CONFIG.update(NEW_CONFIG)

In [ ]:
from rfsoc_rfdc.dsp.ofdm import OFDM
from rfsoc_rfdc.dsp.detection import Detection

In [ ]:
import numpy as np

for atten in np.arange(0.0, 33.0, 3.0):

    for qam_order in ["16QAM", "64QAM", "256QAM", "1024QAM"]:

        for inv_sinc in [0, 1]:
            # Update config
            true_samp_rate = ZCU216_CONFIG['DACSampleRate'] / \
                ZCU216_CONFIG['DACInterpolationRate'] * 1e6
            ZCU216_CONFIG['QAM'] = qam_order
            ZCU216_CONFIG['OFDM_SCHEME'] = OFDM(
                sym_num=100, fft_size=64, sub_num=48, modu=qam_order, cp_rate=0.25)
            ZCU216_CONFIG['DETECTION_SCHEME'] = Detection(
                sample_rate=true_samp_rate)
            ZCU216_CONFIG['InvSincFIR'] = inv_sinc
            ZCU216_CONFIG['OFDM_ATTEN_DB'] = atten
            nco = ZCU216_CONFIG['DACNCO']

            ZCU216_CONFIG['CONFIG_NAME'] = "TxRx_1250M_BW" + \
                "_" + qam_order + "_" + \
                str(nco) + "_" + str(atten) + "_" + str(inv_sinc)

            rfdc_t = RfdcTask(ol)
            led_t = BlinkLedTask(ol)
            tx_t = SingleChTxTask(ol)
            rx_t = SingleChRxTask(ol)

            for task in [rfdc_t]:
                task.start()
                task.join()

            parallel_task = [led_t, tx_t, rx_t]
            for task in parallel_task:
                task.start()

            print(
                f"qam_order {qam_order}, nco {nco}, atten {atten}, inv_sinc {inv_sinc}")

            time.sleep(3)

            for task in parallel_task:
                task.stop()
